In [6]:
# Лабораторная работа 3 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель рекуррентной сети с цепью нейросетевых моделей управляемых рекуррентных блоков 
# с логарифмической функцией активации (гиперболический арксинус) выходного сигнала на скрытом слое
#
# 24.12.24
# Данный файл содержит код построения графиков различных зависимостей

In [1]:
import numpy as np
from gru import (
    GRUModel,
    fibonacci_generator,
    one_by_n_generator,
    plus_one_minus_one_generator,
    squared_generator,
    half_generator,
    arithmetic_progression,
    create_sliding_window_batches,
    one_half_generator,
    sin_generator,    
    mape
)
from gru_with_adam import GRUAdam
import plotly.graph_objects as go
from math import pi

In [2]:
def scale_sequence(sequence):
    mean = np.mean(sequence)
    deviation = np.std(sequence)    
    scaled = (np.array(sequence) - mean) / deviation    
    return scaled, mean, deviation

def descale_sequence(scaled, mean, deviation):
    descaled = scaled * deviation + mean
    return descaled

def predict_next_n_elements(sequence, window_size, batch_size, hidden_size, n,
                            max_epochs, verbosity, lr, use_adam: bool = False):
    scaled, mean, deviation = scale_sequence(sequence)        
    # scaled = sequence  
    X, y = create_sliding_window_batches(scaled, window_size, batch_size, n)
    if not use_adam:
        model = GRUModel(window_size, hidden_size, n)
    else:
        model = GRUAdam(window_size, hidden_size, n)
    model.train(X, y, lr, max_epochs, verbosity=verbosity)
    train_pred, _, _ = model.forward(X)
    training_mape = mape(y, train_pred)           
    x_last = np.vstack(tuple(scaled[-window_size:] for _ in range(batch_size)))
    x_last = np.expand_dims(x_last, axis=0)    
    pred, _, _ = model.forward(x_last)
    descaled_pred = descale_sequence(pred[-1][-1], mean, deviation)
    # descaled_pred = pred[-1][-1]
    return training_mape, model, descaled_pred, mean, deviation

In [10]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
epochs = list(range(100, 2100, 100))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 20
output_size = 3

verbosity = 100000
lr = 5e-3
use_adam = False

sequence = list(sin_generator(seq_length + output_size, alpha=10, beta=pi/4, omega=pi/6))

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)
            

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 100/100, Loss: 0.884858
MAPE: 0.905306
TRAINING FINISHED
Epoch 100/100, Loss: 0.769593
MAPE: 0.840057
TRAINING FINISHED
Epoch 100/100, Loss: 0.807190
MAPE: 0.842075
TRAINING FINISHED
Epoch 100/100, Loss: 0.794541
MAPE: 0.850727
TRAINING FINISHED
Epoch 100/100, Loss: 0.831966
MAPE: 0.832133
TRAINING FINISHED
Epoch 200/200, Loss: 0.616038
MAPE: 0.757940
TRAINING FINISHED
Epoch 200/200, Loss: 0.652556
MAPE: 0.721981
TRAINING FINISHED
Epoch 200/200, Loss: 0.749466
MAPE: 0.782345
TRAINING FINISHED
Epoch 200/200, Loss: 0.638106
MAPE: 0.703709
TRAINING FINISHED
Epoch 200/200, Loss: 0.547271
MAPE: 0.704182
TRAINING FINISHED
Epoch 300/300, Loss: 0.496421
MAPE: 0.659162
TRAINING FINISHED
Epoch 300/300, Loss: 0.501160
MAPE: 0.626691
TRAINING FINISHED
Epoch 300/300, Loss: 0.450271
MAPE: 0.615677
TRAINING FINISHED
Epoch 300/300, Loss: 0.343756
MAPE: 0.524703
TRAINING FINISHED
Epoch 300/300, Loss: 0.444580
MAPE: 0.593911
TRAINING FINISHED
Epoch 400/400, Loss: 0.392701
MAPE: 0

In [12]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [49]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
hidden_sizes = list(range(2, 22, 2))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 20
window_size = 3
batch_size = 2
output_size = 3

verbosity = 100000
lr = 5e-3
max_epochs = 500
use_adam = False

sequence = list(sin_generator(seq_length + output_size, alpha=10, beta=pi/4, omega=pi/6))

for size in hidden_sizes:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)        
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 500/500, Loss: 0.851771
MAPE: 0.898832
TRAINING FINISHED
Epoch 500/500, Loss: 0.918061
MAPE: 1.026745
TRAINING FINISHED
Epoch 500/500, Loss: 0.809541
MAPE: 0.994965
TRAINING FINISHED
Epoch 500/500, Loss: 0.843528
MAPE: 0.934266
TRAINING FINISHED
Epoch 500/500, Loss: 0.570121
MAPE: 0.769079
TRAINING FINISHED
Epoch 500/500, Loss: 0.598965
MAPE: 0.849153
TRAINING FINISHED
Epoch 500/500, Loss: 0.344979
MAPE: 0.664488
TRAINING FINISHED
Epoch 500/500, Loss: 0.406819
MAPE: 0.598202
TRAINING FINISHED
Epoch 500/500, Loss: 0.601636
MAPE: 0.770695
TRAINING FINISHED
Epoch 500/500, Loss: 0.483632
MAPE: 0.824517
TRAINING FINISHED
Epoch 500/500, Loss: 0.317030
MAPE: 0.607575
TRAINING FINISHED
Epoch 500/500, Loss: 0.284266
MAPE: 0.682829
TRAINING FINISHED
Epoch 500/500, Loss: 0.382046
MAPE: 0.669237
TRAINING FINISHED
Epoch 500/500, Loss: 0.404099
MAPE: 0.648871
TRAINING FINISHED
Epoch 500/500, Loss: 0.258333
MAPE: 0.581794
TRAINING FINISHED
Epoch 500/500, Loss: 0.291861
MAPE: 0

In [50]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=hidden_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>количества нейронов на скрытом слое',
    font=dict(size=20),
    xaxis=dict(title='Количество нейронов на скрытом слое'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [86]:
# MAPE relation with sliding window
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()


all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 500
window_sizes = list(range(2, seq_length // 2, 20))
hidden_size = 10
batch_size = 2
output_size = 3

verbosity = 100000
lr = 1e-2
max_epochs = 100
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for size in window_sizes:        

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)              
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 100/100, Loss: 0.033831
MAPE: 0.150670
TRAINING FINISHED
Epoch 100/100, Loss: 0.047728
MAPE: 0.202741
TRAINING FINISHED
Epoch 100/100, Loss: 0.033706
MAPE: 0.220421
TRAINING FINISHED
Epoch 100/100, Loss: 0.086796
MAPE: 0.273037
TRAINING FINISHED
Epoch 100/100, Loss: 0.190484
MAPE: 0.425041
TRAINING FINISHED
Epoch 100/100, Loss: 0.019447
MAPE: 0.502039
TRAINING FINISHED
Epoch 100/100, Loss: 0.015332
MAPE: 0.295300
TRAINING FINISHED
Epoch 100/100, Loss: 0.017207
MAPE: 0.418973
TRAINING FINISHED
Epoch 100/100, Loss: 0.022240
MAPE: 0.478391
TRAINING FINISHED
Epoch 100/100, Loss: 0.023443
MAPE: 0.499902
TRAINING FINISHED
Epoch 100/100, Loss: 0.019605
MAPE: 0.522542
TRAINING FINISHED
Epoch 100/100, Loss: 0.021167
MAPE: 0.516021
TRAINING FINISHED
Epoch 100/100, Loss: 0.021966
MAPE: 0.662980
TRAINING FINISHED
Epoch 100/100, Loss: 0.024505
MAPE: 0.560169
TRAINING FINISHED
Epoch 100/100, Loss: 0.035344
MAPE: 0.570950
TRAINING FINISHED
Epoch 100/100, Loss: 0.034649
MAPE: 1

In [87]:
# Plotting results
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=window_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>размера скользящего окна',
    font=dict(size=20),
    xaxis=dict(title='Размер скользящего окна'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [29]:
# Настройки эксперимента
experiments_amount = 5
seq_length = 10
window_size = 3
batch_size = 2
hidden_size = 10
output_size = 3
max_epochs = 2000
verbosity = 100000
lr = 1e-2

# Словарь с генераторами
sequence_generators = {
    "1, 1, 2, 3, 5, 8,...": lambda n: list(fibonacci_generator(n)),
    "1, 4, 9, 16,...": lambda n: list(squared_generator(n)),
    "1, 2, 3, 4, 5,...": lambda n: list(arithmetic_progression(n, a0=1, d=1)),    
    "1, 0.5 ,0.(3), 0.25, 0.2,...": lambda n: list(one_by_n_generator(n)),
    "1, -1, 1, -1,...": lambda n: list(plus_one_minus_one_generator(n)),
    "1, 0.5, 1, 0.5, 0.5, 1,...": lambda n: list(one_half_generator(n)),
}

all_mapes = []
mapes_mean = []
all_sequences = []

# Тренировка для каждой последовательности
for seq_name, seq_generator in sequence_generators.items():
    sequence = seq_generator(seq_length + output_size)
    print(sequence)   

    mapes_sum = 0
    for _ in range(experiments_amount):
        all_sequences.append(seq_name)
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)

    mapes_mean.append(mapes_sum / experiments_amount)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.013494
MAPE: 0.413876
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.016975
MAPE: 0.474967
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.011370
MAPE: 0.373764
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.016158
MAPE: 0.467851
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.016539
MAPE: 0.441455
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.003922
MAPE: 0.197437
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.004621
MAPE: 0.201097
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.008123
MAPE: 0.258518
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.006724
MAPE: 0.265829
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.008951
MAPE: 0.300480
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.004778
MAPE: 0.148969
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.006750
MAPE: 0.175396
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.002131
MAPE: 0.099188
TRAINING F

In [30]:
# Построение графика
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sequences, y=all_mapes, name="Все наблюдения", mode="markers")
)
fig.add_trace(
    go.Scatter(x=list(sequence_generators.keys()), y=mapes_mean, name="Средние значения", mode="markers")
)
fig.update_layout(
    title="Зависимость средней абсолютной ошибки<br>от вида последовательности",
    font=dict(size=20),
    xaxis=dict(title="Вид последовательности"),
    yaxis=dict(title="Средняя абсолютная ошибка"),
    width=1280,
    height=720
)
fig.show()

In [18]:
# Arithmetic
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(arithmetic_progression(seq_length, 1, 1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")  

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.017820
MAPE: 0.008936
TEST
Предсказания на тестовой выборке:
 [18.09476853 19.10482458 20.19017522]
Истинное значение на тестовой выборке:
 [18 19 20]
MAPE: 0.6764%


In [19]:
# Fibonacci
seq_length = 15
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(fibonacci_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 1368.325603
MAPE: 0.456188
TEST
Предсказания на тестовой выборке:
 [125.56325952 203.17306437 328.7341584 ]
Истинное значение на тестовой выборке:
 [144 233 377]
MAPE: 12.8024%


In [20]:
# 1 / n
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(one_by_n_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.001371
MAPE: 0.286088
TEST
Предсказания на тестовой выборке:
 [0.06865221 0.06474164 0.07640989]
Истинное значение на тестовой выборке:
 [0.05555556 0.05263158 0.05      ]
MAPE: 33.1343%


In [21]:
# 1, -1, 1, -1, ...
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(plus_one_minus_one_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 2000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

Epoch 2000/2000, Loss: 0.081951
MAPE: 0.199536
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.081933
MAPE: 0.199518
TEST
Предсказания на тестовой выборке:
 [-0.26292435  0.18949469 -0.21762125]
Истинное значение на тестовой выборке:
 [-1  1 -1]
MAPE: 77.6653%


In [22]:
# n ** 2
seq_length = 10
window_size = 3
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(squared_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.275702
MAPE: 0.010133
TEST
Предсказания на тестовой выборке:
 [ 64.46419645  81.50586423 100.43683612]
Истинное значение на тестовой выборке:
 [ 64  81 100]
MAPE: 0.5956%
